In [1]:
import damask
import numpy as np
import pyvista as pv
pv.set_jupyter_backend('static')

In [2]:
geom = '2cubic_texture_elongated_grains.vti'       # path for geometry file
material_config = 'material.yaml'    # path for material.yaml

In [3]:
v = damask.VTK.load(geom)
material_ID = v.get('material').flatten()

In [4]:
ma = damask.ConfigMaterial.load(material_config)

phases = list(ma['phase'].keys())
info = []

for m in ma['material']:
    c = m['constituents'][0]
    phase = c['phase']
    info.append({'phase':   phase,
                 'phaseID': phases.index(phase),
                 'lattice': ma['phase'][phase]['lattice'],
                 'O':       c['O'],
                })

In [5]:
l = np.array([0,0,1])                            # lab frame direction for IPF

IPF = np.ones((len(material_ID),3),np.uint8)
for i,data in enumerate(info):
    IPF[np.where(material_ID==i)] = \
    np.uint8(damask.Orientation(data['O'],lattice=data['lattice']).IPF_color(l)*255)

In [6]:
v = v.set(f'IPF_{l}',IPF)

p   = np.array([d['phase'] for d in info])
pid = np.array([d['phaseID'] for d in info])
v = v.set('phase',p[material_ID],'phase')
v = v.set('phaseID',pid[material_ID],'phaseID')

v.save('2cubic_texture_equiaxed_damask.vti_IPF+phase')

In [7]:
#q = np.array([0,1,1])                            # lab frame direction for IPF
#
#IPFm = np.ones((len(material_ID),3),np.uint8)
#for i,data in enumerate(info):
#    IPFm[np.where(material_ID==i)] = np.uint8(damask.Orientation(data['O'],lattice=data['lattice']).IPF_color(q)*255)

In [8]:
#t = v.set(f'IPFm_{q}',IPF)
#w   = np.array([d['phase'] for d in info])
#wid = np.array([d['phaseID'] for d in info])
#t = v.set('phase',w[material_ID],'phase')
#t = v.set('phaseID',wid[material_ID],'phaseID')
#t.save('cubic_texture_elongated_grains_initial_IPFq+phase')